# LAB 02.03 - Pandas

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/fagonzalezo/ai4eng-unal/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()


In [ ]:
from local.lib.rlxmoocapi import submit, session
session.LoginSequence(endpoint=init.endpoint, course_id=init.course_id, lab_id="L02.03", varname="student");

In [3]:
import numpy as np
import pandas as pd

## Task 1: Extract data

we have a dataframe of items with a price and, **sometimes**, with an extra column (`margin`).

You will have to complete a function that will **filter** the dataframe selecting the rows:

- whose price is > 100
- **OR** whose margin is >10, if the column `margin` is present

your function must return **A LIST** with the item ids of the selected rows

For instance, with the following dataframe

             price  category
    itemid                  
    56556   108.15       2.0
    73065    83.54       2.0
    36619   114.42       0.0
    73414    82.94       0.0
    13410   115.13       2.0
    66153    91.50       1.0
    77380    85.82       0.0
    73249    95.95       0.0
    11654   100.22       2.0
    11972    77.16       1.0

your must return the following list:

     [56556, 36619, 13410, 11654]
     
But if you get the following dataframe

             price  category  margin
    itemid                          
    39059    98.11       0.0   11.04
    19526    98.11       1.0   11.25
    78176    94.34       1.0   10.51
    50948   102.37       1.0   10.77
    12111    98.07       1.0    8.50
    56191    98.53       1.0   11.65
    38887    91.49       2.0   11.24
    77915   117.30       0.0    8.64
    55010    96.13       0.0    8.95
    45925    98.59       1.0   10.45
    
you must return the following list

    [39059, 19526, 78176, 50948, 56191, 38887, 77915, 45925]

In [4]:
def create_df(missing=False, n=10):
    itemid   = np.random.randint(100000, size=n)+1000
    category = np.random.randint(3, size=n)
    price    = np.round(np.random.normal(loc=100, scale=10, size=n),2)
    margin   = np.round(np.random.normal(loc=10, scale=1, size=n),2)
    
    if missing:
        nmissing = np.random.randint(len(price)//2)+2                                     
        price[np.random.permutation(len(price))[:nmissing]] = np.nan
    
    d = pd.DataFrame(np.r_[[price, category, margin]].T, index=itemid, columns=["price", "category", "margin"])
    d.index.name="itemid"
    if np.random.random()>.5:
        d = d[d.columns[:2]]
        
    return d

In [5]:
d = create_df()
d

,price,category
itemid,,
56535,99.73,2.0
16922,96.29,1.0
98895,92.06,2.0
80997,106.99,1.0
25570,100.48,0.0
19453,97.66,0.0
36286,104.89,0.0
86326,88.11,2.0
80609,94.20,1.0


In [6]:
def select_items(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    
    if df.columns.__contains__('margin'): df = df[(df['price'] > 100) | (df['margin'] > 10)]
    else: df = df[df['price'] > 100]
    
    result = list(df.index)
    
    return result

**manually check your answer**

In [7]:
d = create_df()
d

,price,category,margin
itemid,,,
47032,88.08,1.0,11.12
70338,94.27,2.0,10.30
45862,89.47,0.0,9.18
42207,92.63,2.0,11.33
58635,96.12,2.0,9.87
95287,98.56,1.0,9.68
9765,103.18,2.0,10.04
87855,85.76,2.0,9.29
63941,95.08,1.0,11.23


In [8]:
select_items(d)

[47032, 70338, 42207, 9765, 63941, 26283]

**submit your code**

In [9]:
student.submit_task(globals(), task_id="task_01");

## Task 2: Group statistics

Complete the following function so that it returns a dataframe with the average, max and min **prices** per category. 

For instance, for the following dataframe

             price  category  margin
    itemid                          
    17946    93.85       1.0   10.64
    61190    91.72       1.0    9.76
    39639   100.16       1.0   10.67
    17791   110.44       2.0    9.65
    7333    101.05       1.0    9.69
    77362   122.33       0.0   11.14
    92646   108.13       2.0   10.58
    27797    85.52       2.0   10.88
    31746    97.56       0.0    9.75
    12355   101.04       2.0    9.51
    
you should return the following dataframe

                 media  maximo  minimo
    categoria                         
    0         109.9450  122.33  97.56
    1          96.6950  101.05  91.72
    2         101.2825  110.44  85.52
    
observe that your result
- must not be a **multilevel** columnset.
- the column names and the index name must be **exactly** as in the example.
- the **index** must be of type **int**.

In [10]:
d.describe()

,price,category,margin
count,10.00000,10.000000,10.000000
mean,94.69900,1.500000,10.398000
std,6.03767,0.707107,0.948868
min,85.76000,0.000000,9.180000
25%,90.26000,1.000000,9.727500
50%,94.67500,2.000000,10.170000
75%,97.95000,2.000000,11.202500
max,103.84000,2.000000,11.940000


In [11]:
def get_stats(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    df['categoria'] = df['category'].astype(int)
    result = df.groupby('categoria').agg(
                media=('price', 'mean'),
                maximo=('price', 'max'),
                minimo=('price', 'min')
                
            )
    return result

**manually check your answer**

In [12]:
d = create_df()
d

,price,category
itemid,,
79191,97.94,0.0
55240,100.63,2.0
39043,93.86,1.0
48564,113.06,1.0
14571,77.31,1.0
51046,102.12,0.0
50868,82.27,1.0
11017,95.78,0.0
44936,104.00,2.0


In [13]:
get_stats(d)

,media,maximo,minimo
categoria,,,
0,98.613333,102.12,95.78
1,91.625000,113.06,77.31
2,104.323333,108.34,100.63


**submit your code**

In [14]:
student.submit_task(globals(), task_id="task_02");

## Task 3: Fill in missing data

Fill in the missing data in the **price** column with the following procedure:

- compute the mean and std of the avaialable prices
- sample from a normal distribution with the computed mean and std (see [`np.random.normal`])(https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), as many samples as missing data
- substitute the missing values with the samples

For instance, for this input dataframe:

             price  category  margin
    itemid                          
    18922      NaN       1.0   10.32
    69500   121.25       1.0   10.22
    76442    90.25       1.0   12.60
    33863   106.51       0.0   10.26
    15904    95.87       1.0   11.51
    41946   103.47       2.0    9.85
    85451    93.08       2.0    9.56
    70028   116.68       1.0    9.11
    26860      NaN       2.0    9.71
    12807    91.48       0.0    9.77
    
your solution might be similar to this (not exactly the same as you will be sampling data):


                 price  category  margin
    itemid                              
    18922    97.441188       1.0   10.32
    69500   121.250000       1.0   10.22
    76442    90.250000       1.0   12.60
    33863   106.510000       0.0   10.26
    15904    95.870000       1.0   11.51
    41946   103.470000       2.0    9.85
    85451    93.080000       2.0    9.56
    70028   116.680000       1.0    9.11
    26860   103.294843       2.0    9.71
    12807    91.480000       0.0    9.77
    
    
**WARN**: your function must not modify the original dataframe, make a copy of the input dataframe, fill the values in the copy and return it.

**HINT**: use the [`isna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) method of a dataframe or a series.

In [15]:
d = create_df(missing=True)
d

,price,category
itemid,,
78171,NaN,1.0
30459,113.78,1.0
31467,84.23,1.0
75036,NaN,2.0
77071,NaN,0.0
69278,96.78,1.0
99018,90.43,2.0
99092,115.77,1.0
25839,121.03,2.0


In [16]:
def fillna(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    miu, sigma = df['price'].mean() , df['price'].std()
    n = df['price'].isna().sum()
    df.loc[df['price'].isna(),'price'] = np.random.normal(miu, sigma, n)
    result = df
    return result

**check manually your code**

In [17]:
d = create_df(missing=True)
d

,price,category,margin
itemid,,,
65590,NaN,0.0,12.47
60913,103.91,2.0,11.07
64432,104.37,0.0,9.52
58058,NaN,2.0,10.13
69335,100.44,1.0,10.04
52272,NaN,2.0,9.27
9594,104.12,2.0,10.62
87505,101.72,2.0,10.26
6879,NaN,1.0,11.93


In [18]:
fillna(d)

,price,category,margin
itemid,,,
65590,105.164000,0.0,12.47
60913,103.910000,2.0,11.07
64432,104.370000,0.0,9.52
58058,102.217109,2.0,10.13
69335,100.440000,1.0,10.04
52272,102.443730,2.0,9.27
9594,104.120000,2.0,10.62
87505,101.720000,2.0,10.26
6879,101.957897,1.0,11.93


**submit your code**

In [19]:
student.submit_task(globals(), task_id="task_03");